In [2]:
import folium
from main import main_train, main_train_Q
from viewer import get_position, view
import numpy as np
from get_data import get_spots, get_time

In [3]:
map = folium.Map(location=(34.985849, 135.758767), zoom_start=13)
# プロット
for p in get_position().values():
    folium.Marker(location=p).add_to(map) 
map

In [4]:
pos = get_position()
map = folium.Map(location=(34.985849, 135.758767), zoom_start=13)
d = get_time("嵐山")
spots = get_spots()
lines = []
for i in range(25):
    for j in range(25):
        if i != j and d[i][j] < 10000:
            lines.append({"from": pos[spots[i]], "to": pos[spots[j]]})
# プロット
for p in pos.values():
    folium.Marker(location=p).add_to(map) 
for line in lines:
    folium.PolyLine(
        locations=[line["from"], line["to"]]
    ).add_to(map)
map.save('result/round_map.html')
map

In [5]:
def RGB(*colors):
    """0 <= r,g,b <= 255 を受け取ってカラーコードを返す．"""
    res = "#"
    for c in colors:
        res += ("0" + hex(c)[2:])[-2:]
    return res


In [25]:
def recommend(Q, p, t=0, n=5, zoom_start=13):
    """
     時刻tにおける地点pのQ値を入力
     Qの中で上位5つを図示する．
    """
    rank = np.argsort(-np.array(Q[(p, t)]))
    pos = get_position()
    spots = get_spots()
    s = pos[spots[p]]
    map = folium.Map(location=s, zoom_start=zoom_start)
    # プロット
    # for p in pos.values():
    #     folium.Marker(location=p).add_to(map) 
    lines = []
    for i in range(n):
        lines.append({"from": s, "to": pos[spots[rank[i]]], "weight": 3 if i else 10, "color": RGB(93, 70+150*i//n, 255)})

    for line in lines:
        folium.PolyLine(
            locations=[line["from"], line["to"]],
            weight=line["weight"],
            color=line["color"]
        ).add_to(map)

    return map

In [7]:
unique_results_Q = main_train_Q(num_loop=16, train_loop=10_000_000) #10_000_000
unique_results = main_train(num_loop=16, train_loop=1_000_000) #10_000_000

0h 37m 21s
0h 1m 25s


In [9]:
route = unique_results[0][1]
view(route)

In [1]:
Q = unique_results_Q[0][1]
map = recommend(Q, 12, t=0, n=5, zoom_start=14)
map.save('result/recommend.html')
map

NameError: name 'unique_results_Q' is not defined

In [11]:
import pickle
def save(result, country):
    file_name = "result/{}.pkl".format(country)
    with open(file_name, "wb") as f:
        pickle.dump(result, f)

save(unique_results,"中国嵐山")

In [12]:
country = "中国"
for i in [0, 2, 4,6]:
    if i < len(unique_results):
        route = unique_results[i][1]
        view(route).save('result/{}_{}_add.html'.format(country, i + 1))